# Baixando Bibliotecas

In [ ]:
!pip install pandas

# Importando Bibliotecas

In [ ]:
import pandas as pd

# Extração do Dataset para Testes

In [ ]:
'''
#nome do arquivo
#path para os arquivos de entrada
#path para os arquivos de saida
#colunas a serem salvas
#numero de linhas a serem s
'''
def extract_data(filename, input_path, output_path, columns, number_rows, separator):
    offset = 0
    df = pd.read_csv(
        input_path+filename, 
        encoding = "ISO-8859-1", 
        engine='python', 
        sep=separator, 
        skiprows=offset, 
        nrows=number_rows, 
        usecols=columns
    )
    output_filename = output_path+"SLICED_"+filename
    df.to_csv(output_filename, sep=';', encoding = "ISO-8859-1")

In [ ]:
microdados = {
    "2015":"MICRODADOS_ENEM_2015.csv",
    "2016":"MICRODADOS_ENEM_2016.csv",
    "2017":"MICRODADOS_ENEM_2017.csv",
    "2018":"MICRODADOS_ENEM_2018.csv",
    "2019":"MICRODADOS_ENEM_2019.csv"
}

colunas = [
    "TP_PRESENCA_CN","TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT",
    "CO_PROVA_CN","CO_PROVA_CH","CO_PROVA_LC","CO_PROVA_MT",
    "NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_LC","NU_NOTA_MT",
    "TX_RESPOSTAS_CN","TX_RESPOSTAS_CH","TX_RESPOSTAS_LC","TX_RESPOSTAS_MT",
    "TP_LINGUA","TX_GABARITO_CN","TX_GABARITO_CH","TX_GABARITO_LC",
    "TX_GABARITO_MT","TP_STATUS_REDACAO","NU_NOTA_COMP1","NU_NOTA_COMP2",
    "NU_NOTA_COMP3","NU_NOTA_COMP4","NU_NOTA_COMP5","NU_NOTA_REDACAO","SG_UF_RESIDENCIA","NO_MUNICIPIO_ESC","NO_MUNICIPIO_RESIDENCIA"
]

number_rows = 100 #cem mil linhas

input_path = "/Users/fabio/Desktop/PLE/Mineracao_Dados_Educacionais/microdados/"

output_path = path

extract_data(microdados['2015'], input_path, output_path, colunas, number_rows, ',')
extract_data(microdados['2016'], input_path, output_path, colunas, number_rows, ';')
extract_data(microdados['2017'], input_path, output_path, colunas, number_rows, ';')
extract_data(microdados['2018'], input_path, output_path, colunas, number_rows, ';')
extract_data(microdados['2019'], input_path, output_path, colunas, number_rows, ';')



# Juntando os .CSV das Competências e Habilidades

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

enem2016 = pd.read_csv('drive/My Drive/MDE/microdados_sliced/SLICED_MICRODADOS_ENEM_2016.csv', sep=';')
prova = pd.read_csv('drive/My Drive/MDE/2016/itens_prova_2016.csv', sep=';')

enem2016 = enem2016.loc[:, ~enem2016.columns.str.contains('^Unnamed')]
enem2016 = enem2016.dropna()
enem2016 = enem2016.drop(["TP_PRESENCA_CN", 
                          "TP_PRESENCA_CH", 
                          "TP_PRESENCA_LC", 
                          "TP_PRESENCA_MT", 
                          "CO_PROVA_CN", 
                          "CO_PROVA_LC", 
                          "CO_PROVA_MT", 
                          "NU_NOTA_CN", 
                          "NU_NOTA_CH", 
                          "NU_NOTA_LC",
                          "NU_NOTA_MT",
                          "TX_GABARITO_CN",
                          "TX_GABARITO_CH",
                          "TX_GABARITO_LC",
                          "TX_GABARITO_MT",
                          "TP_STATUS_REDACAO",
                          "NU_NOTA_COMP1",
                          "NU_NOTA_COMP2",
                          "NU_NOTA_COMP3",
                          "NU_NOTA_COMP4",
                          "NU_NOTA_COMP5",
                          "NU_NOTA_REDACAO"], axis=1)

aux = enem2016[['TX_RESPOSTAS_CH', "CO_PROVA_CH"]]


label = "questao_"

ch = prova[prova["SG_AREA"]=="CH"]
ch["CO_PROVA_CH"] = ch.CO_PROVA
ch = ch.drop(["SG_AREA", 
         "CO_ITEM",
         "TX_COR",
         "TP_LINGUA",
         "IN_ITEM_ADAPTADO",
         "CO_PROVA"], axis = 1)


for i in range(1,46):
  aux[[label+str(i)]] = aux.TX_RESPOSTAS_CH.astype(str).str[i-1]
  aux2 = ch[ch.CO_POSICAO == i]
  aux2[label+str(i)+"_habilidade"] = aux2.CO_HABILIDADE
  aux2[label+str(i)+"_gabarito"] = aux2.TX_GABARITO
  aux2 = aux2.drop(["CO_HABILIDADE","CO_POSICAO", "TX_GABARITO"], axis=1)
  aux = aux.merge(aux2, on=['CO_PROVA_CH'])

aux = aux.drop(["TX_RESPOSTAS_CH"], axis=1)

aux[["questao_1", "questao_1_habilidade"]]


,CO_PROVA_CH,questao_1,questao_1_habilidade,questao_1_gabarito,questao_2,questao_2_habilidade,questao_2_gabarito,questao_3,questao_3_habilidade,questao_3_gabarito,questao_4,questao_4_habilidade,questao_4_gabarito,questao_5,questao_5_habilidade,questao_5_gabarito,questao_6,questao_6_habilidade,questao_6_gabarito,questao_7,questao_7_habilidade,questao_7_gabarito,questao_8,questao_8_habilidade,questao_8_gabarito,questao_9,questao_9_habilidade,questao_9_gabarito,questao_10,questao_10_habilidade,questao_10_gabarito,questao_11,questao_11_habilidade,questao_11_gabarito,questao_12,questao_12_habilidade,questao_12_gabarito,questao_13,questao_13_habilidade,questao_13_gabarito,...,questao_32_gabarito,questao_33,questao_33_habilidade,questao_33_gabarito,questao_34,questao_34_habilidade,questao_34_gabarito,questao_35,questao_35_habilidade,questao_35_gabarito,questao_36,questao_36_habilidade,questao_36_gabarito,questao_37,questao_37_habilidade,questao_37_gabarito,questao_38,questao_38_habilidade,questao_38_gabarito,questao_39,questao_39_habilidade,questao_39_gabarito,questao_40,questao_40_habilidade,questao_40_gabarito,questao_41,questao_41_habilidade,questao_41_gabarito,questao_42,questao_42_habilidade,questao_42_gabarito,questao_43,questao_43_habilidade,questao_43_gabarito,questao_44,questao_44_habilidade,questao_44_gabarito,questao_45,questao_45_habilidade,questao_45_gabarito
0,296.0,E,21,E,E,9,E,E,23,B,C,22,C,A,26,B,D,19,C,B,5,E,D,29,D,D,13,D,C,12,B,D,3,D,C,17,D,A,27,A,...,C,E,8,E,D,3,C,A,26,A,B,30,E,C,25,C,B,20,B,B,5,B,E,19,B,A,6,A,D,1,D,C,7,C,C,10,E,B,14,C
1,296.0,D,21,E,E,9,E,B,23,B,C,22,C,D,26,B,D,19,C,E,5,E,D,29,D,D,13,D,D,12,B,D,3,D,D,17,D,A,27,A,...,C,D,8,E,A,3,C,E,26,A,B,30,E,A,25,C,A,20,B,E,5,B,B,19,B,D,6,A,D,1,D,A,7,C,D,10,E,A,14,C
2,296.0,E,21,E,C,9,E,A,23,B,D,22,C,A,26,B,E,19,C,D,5,E,C,29,D,E,13,D,D,12,B,D,3,D,C,17,D,A,27,A,...,C,C,8,E,B,3,C,A,26,A,B,30,E,A,25,C,B,20,B,B,5,B,E,19,B,A,6,A,D,1,D,D,7,C,C,10,E,C,14,C
3,296.0,A,21,E,D,9,E,B,23,B,E,22,C,B,26,B,C,19,C,B,5,E,B,29,D,E,13,D,D,12,B,A,3,D,D,17,D,E,27,A,...,C,B,8,E,B,3,C,B,26,A,B,30,E,A,25,C,B,20,B,E,5,B,A,19,B,B,6,A,B,1,D,C,7,C,C,10,E,D,14,C
4,296.0,E,21,E,B,9,E,E,23,B,E,22,C,B,26,B,E,19,C,E,5,E,D,29,D,C,13,D,E,12,B,D,3,D,D,17,D,A,27,A,...,C,E,8,E,C,3,C,B,26,A,E,30,E,D,25,C,B,20,B,B,5,B,B,19,B,A,6,A,A,1,D,C,7,C,E,10,E,D,14,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71690,339.0,A,28,A,E,15,B,B,2,D,D,9,D,D,6,E,C,5,B,D,24,B,B,12,B,E,4,E,E,17,D,D,29,D,B,22,E,D,3,D,...,B,C,1,E,E,20,E,D,1,C,B,5,B,D,26,D,A,21,D,A,11,A,A,15,A,D,22,D,A,23,E,A,3,E,C,18,C,A,6,B
71691,336.0,C,8,D,D,13,E,B,19,B,E,1,E,D,4,E,D,17,D,D,29,D,B,6,E,B,5,B,D,24,B,B,12,B,B,5,B,A,27,A,...,B,D,11,B,A,3,C,A,18,C,E,24,B,A,7,C,A,23,A,B,5,B,D,26,D,B,21,D,D,11,A,E,23,E,A,3,E,C,18,C
71692,308.0,B,1,D,C,7,C,D,10,E,E,14,C,A,19,B,B,6,A,C,15,A,E,21,A,B,18,D,C,25,C,E,20,B,E,5,B,A,8,E,...,D,E,12,B,A,6,A,B,30,D,C,18,B,E,2,C,A,4,A,A,4,D,B,23,B,C,22,C,D,26,B,E,19,C,A,21,E,B,9,E
71693,308.0,D,1,D,E,7,C,C,10,E,E,14,C,B,19,B,D,6,A,D,15,A,C,21,A,D,18,D,E,25,C,B,20,B,E,5,B,D,8,E,...,D,D,12,B,A,6,A,E,30,D,A,18,B,D,2,C,A,4,A,D,4,D,C,23,B,D,22,C,A,26,B,B,19,C,C,21,E,B,9,E


In [ ]:
enem2015.describe()

,Unnamed: 0,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
count,100000.000000,99997.000000,99997.000000,99997.000000,99997.000000,79136.000000,79136.000000,77385.000000,77385.000000,79136.000000,79136.000000,77385.000000,77385.000000,100000.000000,99997.000000,99983.000000,99983.000000,99983.000000,99983.000000,99983.000000,99983.000000
mean,49999.500000,0.792244,0.792244,0.774833,0.774833,236.489297,232.490548,240.478439,244.477121,491.013602,575.815307,524.802009,483.333466,0.552520,1.717962,97.537541,92.098227,89.798266,93.892112,67.208725,440.534871
std,28867.657797,0.406762,0.406762,0.418842,0.418842,1.493783,1.507962,1.499579,1.489859,72.182337,66.798087,67.771127,110.235158,0.497236,1.694627,57.891790,56.154533,55.660089,57.866770,52.258470,267.573942
min,0.000000,0.000000,0.000000,0.000000,0.000000,235.000000,231.000000,239.000000,243.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24999.750000,1.000000,1.000000,1.000000,1.000000,235.000000,231.000000,239.000000,243.000000,438.700000,536.600000,483.200000,401.600000,0.000000,1.000000,80.000000,60.000000,60.000000,80.000000,0.000000,340.000000
50%,49999.500000,1.000000,1.000000,1.000000,1.000000,236.000000,232.000000,240.000000,244.000000,485.100000,581.500000,529.100000,462.800000,1.000000,1.000000,120.000000,120.000000,100.000000,120.000000,80.000000,520.000000
75%,74999.250000,1.000000,1.000000,1.000000,1.000000,237.000000,233.000000,241.000000,245.000000,535.600000,620.700000,571.100000,543.500000,1.000000,1.000000,140.000000,120.000000,120.000000,120.000000,100.000000,620.000000
max,99999.000000,2.000000,2.000000,2.000000,2.000000,278.000000,274.000000,282.000000,286.000000,850.500000,837.400000,774.400000,1003.100000,1.000000,98.000000,200.000000,200.000000,200.000000,200.000000,200.000000,1000.000000


# Extraindo dos .CSV os Dados de Pernambuco

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
microdados = pd.read_csv("drive/My Drive/MDE/2016/microdados_enem_2016.csv",sep=';', encoding = "ISO-8859-1", usecols=colunas)


In [ ]:
microdados_pe = microdados.loc[microdados['SG_UF_RESIDENCIA']=='PE'].dropna()
microdados_pe = microdados_pe.drop(["SG_UF_RESIDENCIA"], axis=1)
microdados_pe.to_csv("drive/My Drive/MDE/2016/MICRODADOS_ENEM_2016_PE.csv")
microdados_pe

,NO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_ESC,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
116,Recife,Recife,1.0,1.0,1.0,1.0,294.0,298.0,301.0,305.0,611.4,602.6,612.5,558.6,ECAEDACBCAAEDAEBDABDEDAABBCADBBACACDBDBCBDCCE,BDDDDDACEBECDCCBBEDBDDCEEADECCADAAEEDACACBABD,EABBA99999DAAACDECBCDABEAABBEEEBAEEBABEADEEAEA...,E..EBCCCCBAEBCECABCDBBBBCBBBABBAADBBDBBCBABDA,0,EADBDCCBADBEBCEBCABDEBEEBECADDBEACACDDBCADCCA,EDDBDDACDCEDDACBBBCBCDCECADBEEBAAADECAECCBCAD,EECDABBECCDAAECBBCBDABBAAEEBBDCBADEBABEADCECEB...,CCEEBCCEBAABDDECADCEECDBCBDBECBADDACEAABDBDAD,1.0,160.0,180.0,200.0,160.0,160.0,860.0
397,São Lourenço da Mata,São Lourenço da Mata,1.0,1.0,1.0,1.0,353.0,358.0,361.0,366.0,419.4,419.7,501.4,338.5,EBDAAEDCDBCACCEBDCADCCDAAEDAEACDEAADCACBDAEDE,ECBBDDBAECEAACBDEAAABEDECAEBBCCABBAEDCDBACCDE,ABDDA99999DBAACBECBBCDCDDEACDBDCACDBCEEBEDBAAD...,DCBAEABACBABADCBDCBADEBABCABCBBDBDBCECEADBADC,0,EDABDEBBCAEBXBACEBEACCCACCDBCBDACDEDECCACCAAD,EECBDDACBCABBCEAAADEEEDCABDDECBBABEBBBEDDEBED,ABCDDBCACDDBDAEBECABACCCDDACEEACAEAECCECCDAEBD...,BDEBBCAACCBADCCACEDCEBCBDDDEEEEBCEADCDCDBABBD,1.0,120.0,120.0,100.0,100.0,120.0,560.0
1339,Recife,Jaboatão dos Guararapes,1.0,1.0,1.0,1.0,291.0,295.0,302.0,306.0,526.0,594.4,582.9,490.8,BCABEADADDBAACDEABCCAADECCDABCEBCDEBBCDACECBA,EABDABDAEBBDDDECACADCCACDBEAACECDBDAEAABADDAD,DAECA99999DEBEAEAACBACBDBBDCAAEDEAABADCDABBDEB...,CCEAEBBAAABABBCCAECCABABBADBEDCBADAAEEDBCCDAB,0,BCADBCEDCCBADBDCADBEBDBEADCCAEACDBEDBCACAEBEE,BCBCCADCCBEDDBEEADBDCECCBBECAEEDDADCDDACAADBA,EDECACCBBEDEBBBDABCBBCCEBADCEBAAEAAEDCBCABDBAB...,CEECDBDADEBCCEBABECEACBDABDBDDABDDECADCCEACBA,1.0,160.0,120.0,120.0,160.0,160.0,720.0
1483,Ribeirão,Ribeirão,1.0,1.0,1.0,1.0,293.0,297.0,300.0,304.0,460.4,509.1,489.8,374.0,DBBACEDECABEBDCCDCBADCCCEDAADDACCDACEBCDBAEBC,DCEACABEDACBCDCDEBEADCDBDECBC.BCDCACBADCADACA,BACCD99999DAACBCBEEADBDACEBECBCEEADADCACEEBDEA...,BCACDABCDADCDCCBACDACEBEBDB.DDCADCECADBEBBDCA,0,EBEECADBCABECDDBEADCCABDEABEBCADBADDCCCEDBBCA,DCECBAAADCBBECAEDCEDDACCBDDACEDDBADBCADBCBCEE,DECEACCEBBDADEBABEAAEDCBCABDCEBADCEBAAECBBCBDA...,CBAACCEABDDCDBDDCEAECADBECEBACBDABDBEBCDADCEE,1.0,100.0,100.0,80.0,100.0,0.0,380.0
2125,Recife,Recife,1.0,1.0,1.0,1.0,293.0,297.0,301.0,305.0,585.2,569.0,558.9,487.9,BDEEADECCEDDABDCEADDAADDEEBEACBBBCBDABECBECBD,DCEABDAADBABACEBACEADCCBADBBCBBECACBBADEECEDE,EDBAC99999CAAECABBCDAEAACDEEBAEBAEBBDBBABAACAA...,DEBCEBDDAAAECDEAEECADDBCBAEADBDACBAADDBBECBDD,0,EBEECADBCABECDDBEADCCABDEABEBCADBADDCCCEDBBCA,DCECBAAADCBBECAEDCEDDACCBDDACEDDBADBCADBCBCEE,EECDABBECCDAAECBBCBDABBAAEEBBDCBADEBABEADCECEB...,CCEEBCCEBAABDDECADCEECDBCBDBECBADDACEAABDBDAD,1.0,120.0,120.0,120.0,120.0,120.0,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8626918,Jaboatão dos Guararapes,Recife,1.0,1.0,1.0,1.0,292.0,296.0,301.0,305.0,436.2,402.3,407.2,494.1,BACBDDAECADEBEACDAAACBDCBACACECBCAEDCBACDCECB,ACACCBEEECDBCCEBEBABBABBCBEDEDCCBCBEEDEDABCEC,CDDCB99999CEDAECBCECDEAEBDBDAAAABABACBABAADCBC...,EDBBDBEEDCBBCCCCCBDCEECBCDADCEBCDBEDCBDBEACBD,0,DBDCCACADBADBCABDEBEECECDDBCADCCBEBBEDBEAEACA,EEBCBCEDDBDDACCCBCADADBAADEDDADCECAECBBBADCEC,EECDABBECCDAAECBBCBDABBAAEEBBDCBADEBABEADCECEB...,CCEEBCCEBAABDDECADCEECDBCBDBECBADDACEAABDBDAD,6.0,0.0,0.0,0.0,0.0,0.0,0.0
8626996,Carpina,Carpina,1.0,1.0,1.0,1.0,292.0,296.0,300.0,304.0,481.1,530.2,572.9,417.2,CBDCEAADCADAABECDCECACBDCBBCADBDBDCAABCECEDAE,BBCDEABBCDBDADBCABACEEBAEEEBDEACADAACBBBBBEEB,99999DCACDAABBCABDAACDDCEBADCEBBBCECDACCBBCACA...,DDACCAAEECBACEDEBADBDDACBCBCCEEDCDBEDADCECCDD,1,DBDCCACADBADBCABDEBEECECDDBCADCCBEBBEDBEAEACA,EEBCBCEDDBDDACCCBCADADBAADEDDADCECAECBBBADCEC,DECEACCEBBDADEBA